In [71]:
import geoengine as ge


In [73]:
ge.initialize("http://localhost:3030/api")#, credentials=("admin@localhost", "admin"))


In [74]:
dataset_name_prefix = "sentinel2_nrw_crop_10m"
dataset_info = {
    'b02': {'dataset': '48d30acd-a378-4f2a-89b3-d73430c0f29e', 'upload': 'd46c7bba-1db3-4481-9a61-6f8cff0ca40d', 'dtype': "U16", 'description': "Blue band", 'min': 0, 'max': 15000},
    'b03': {'dataset': '200ed06c-fcd1-41a3-b2dd-68b5d0b338a9', 'upload': 'e28aa6c4-dc80-440e-a730-eeed99a61205', 'dtype': "U16", 'description': "Green band", 'min': 0, 'max': 15000},
    'b04': {'dataset': 'b9ececef-5a0c-4294-be16-334e8017f60f', 'upload': '5c05fc58-a15a-4e98-97ca-fdbaff0c9946', 'dtype': "U16", 'description': "Red band", 'min': 0, 'max': 15000},
    'b08': {'dataset': '8e635367-35ff-42f9-bb21-88ae9db8be8d', 'upload': '03632160-a912-47de-a14d-f9eb8f6ba09f', 'dtype': "U16", 'description': "NIR band", 'min': 0, 'max': 15000},
    'scl': {'dataset': '64636605-10da-4576-bf21-a265cc1f7d9c', 'upload': '263a4410-2ca4-4f29-adaa-a7082d00d5ab', 'dtype': "U8", 'description': "SCL : Scene Classification Layer", 'min': 0, 'max': 15},
}

In [75]:
from pathlib import Path
from datetime import datetime

def files_with_time(path_to_volume, path_in_volume, data_folder):
    upload_path = Path(path_to_volume, path_in_volume, data_folder)
    sorted_files_list = sorted(upload_path.glob("*.tiff"))
    file_names_list = [file.name for file in sorted_files_list]
    file_date_list = [datetime.strptime(file[7:30], "%Y-%m-%dT%H-%M-%S-%f") for file in file_names_list]
    
    return file_names_list, file_date_list

files_with_time("/home/droenner/git/geoengine/test_data", "sentinel2", dataset_name_prefix+"_b02")

(['raster_2021-01-02T10-46-31-000.tiff',
  'raster_2021-01-02T10-46-45-000.tiff',
  'raster_2021-01-04T10-36-32-000.tiff',
  'raster_2021-01-04T10-36-47-000.tiff',
  'raster_2021-01-07T10-46-29-000.tiff',
  'raster_2021-01-07T10-46-44-000.tiff',
  'raster_2021-01-09T10-36-34-000.tiff',
  'raster_2021-01-09T10-36-49-000.tiff',
  'raster_2021-01-12T10-46-31-000.tiff',
  'raster_2021-01-12T10-46-45-000.tiff',
  'raster_2021-01-14T10-36-33-000.tiff',
  'raster_2021-01-14T10-36-48-000.tiff',
  'raster_2021-01-17T10-46-30-000.tiff',
  'raster_2021-01-17T10-46-44-000.tiff',
  'raster_2021-01-19T10-36-34-000.tiff',
  'raster_2021-01-19T10-36-49-000.tiff',
  'raster_2021-01-22T10-46-31-000.tiff',
  'raster_2021-01-22T10-46-45-000.tiff',
  'raster_2021-01-24T10-36-34-000.tiff',
  'raster_2021-01-24T10-36-48-000.tiff',
  'raster_2021-01-27T10-46-30-000.tiff',
  'raster_2021-01-27T10-46-44-000.tiff',
  'raster_2021-01-29T10-36-34-000.tiff',
  'raster_2021-01-29T10-36-48-000.tiff',
  'raster_2021-0

In [76]:
from datetime import datetime, timedelta

def generate_add_dataset(path_to_volume, path_in_volume, dataset_name, info): 
    rasterband_channel = 1
    origin_x = 421398.306883941579144
    origin_y = 5727832.747780239209533
    origin = [origin_x, origin_y]
    x_pixel_size = 10
    y_pixel_size = -10
    width = 5481 
    height = 4676
    spatial_reference = "EPSG:32632"

    description = info['description']
    min_value = info['min']
    max_value = info['max']

    d_type = info['dtype']
    dataset_id = info['dataset']

    # dataset 
    
    add_dataset_properties = ge.DatasetProperties( 
        dataset_id = ge.datasets.DatasetId(dataset_id),
        name = dataset_name,
        source_operator = "GdalSource",
        description = description,
        symbology = ge.RasterSymbology(
            opacity = 1.0,
            colorizer = ge.LinearGradientColorizer(
            breakpoints=[
                ge.ColorBreakpoint(value=min_value, color=(0, 0, 255, 255)),
                ge.ColorBreakpoint(value=min_value + (max_value-min_value) / 2, color=(0, 255, 0, 255)),
                ge.ColorBreakpoint(value=max_value, color=(255, 0, 0, 255))
            ],
            no_data_color= (0, 0, 0, 0),
            over_color= (0, 0, 0, 0),
            under_color= (0, 0, 0, 0)
            )
        ),
        provenance=[ge.Provenance(
            citation="TODO",
            uri="TODO",
            license="TODO"
        )],
    )

    # gdal
    result_descriptor_measurement = ge.ContinuousMeasurement(
        measurement="refl",
        unit="x"
    )

    result_descriptor = ge.RasterResultDescriptor(
        d_type,
        result_descriptor_measurement,
        spatial_reference,
        spatial_bounds=ge.SpatialPartition2D(origin_x,  origin_y + height * y_pixel_size, origin_x + width * x_pixel_size, origin_y),
        spatial_resolution=ge.SpatialResolution(abs(x_pixel_size), abs(y_pixel_size))
    )

    file_not_found_handling = ge.api.FileNotFoundHandling.NODATA

    geo_transform = ge.api.GdalDatasetGeoTransform(
        originCoordinate = origin, 
        xPixelSize = x_pixel_size, 
        yPixelSize = y_pixel_size
    )

    file_list, file_date_start_list = files_with_time(path_to_volume, path_in_volume, dataset_name)
    file_data_end_list = file_date_start_list[1:] + [file_date_start_list[-1] + timedelta(seconds=5)]


    temporal_params_list = []

    for file_path, (start_date, end_date) in zip(file_list, zip(file_date_start_list, file_data_end_list)):
            # per file
        time_interval = ge.TimeInterval(
            start = start_date,
            end = end_date,
        ).to_api_dict()
        
        gdal_params = ge.api.GdalDatasetParameters(
            filePath = path_in_volume + "/" + dataset_name + "/" + file_path,
            rasterbandChannel = rasterband_channel, 
            geoTransform = geo_transform,
            width = width, 
            height = height , 
            fileNotFoundHandling = file_not_found_handling, 
            noDataValue = None, 
            propertiesMapping = None, 
            gdalOpenOptions = None, 
            gdalConfigOptions = None, 
            allowAlphabandAsMask = True
        )
        
        ts = ge.api.GdalLoadingInfoTemporalSlice(
            time = time_interval,
            params = gdal_params
        )

        temporal_params_list.append(ts)


    meta_data = ge.api.GdalMetaDataList(
    type = "GdalMetaDataList",    
    resultDescriptor = result_descriptor.to_api_dict(),
    params = temporal_params_list, 
    )

    return add_dataset_properties, meta_data


generate_add_dataset("/home/droenner/git/geoengine/test_data/", "sentinel2", dataset_name_prefix + "_b04", dataset_info['b04'])

(<geoengine.datasets.DatasetProperties at 0x7f75a2fd5ed0>,
 {'type': 'GdalMetaDataList',
  'resultDescriptor': {'type': 'raster',
   'dataType': 'U16',
   'measurement': {'type': 'continuous', 'measurement': 'refl', 'unit': 'x'},
   'spatialReference': 'EPSG:32632',
   'time': None,
   'bbox': {'upperLeftCoordinate': {'x': 421398.3068839416,
     'y': 5727832.747780239},
    'lowerRightCoordinate': {'x': 476208.3068839416, 'y': 5681072.747780239}},
   'resolution': {'x': 10, 'y': 10}},
  'params': [{'time': {'start': '2021-01-02T10:46:31.000+00:00',
     'end': '2021-01-02T10:46:45.000+00:00'},
    'params': {'filePath': 'sentinel2/sentinel2_nrw_crop_10m_b04/raster_2021-01-02T10-46-31-000.tiff',
     'rasterbandChannel': 1,
     'geoTransform': {'originCoordinate': [421398.3068839416,
       5727832.747780239],
      'xPixelSize': 10,
      'yPixelSize': -10},
     'width': 5481,
     'height': 4676,
     'fileNotFoundHandling': <FileNotFoundHandling.NODATA: 'NoData'>,
     'noDataValu

In [77]:
volume = ge.volumes()[0]

datasets = {}

for name, info in dataset_info.items():
    dataset_name = dataset_name_prefix + "_" + name
    add_dataset_properties, meta_data = generate_add_dataset("/home/droenner/git/geoengine/test_data/", "sentinel2/", dataset_name, info)
    dataset = ge.add_dataset(volume, add_dataset_properties, meta_data)
    print(dataset)
    datasets[name] = dataset

datasets

48d30acd-a378-4f2a-89b3-d73430c0f29e
200ed06c-fcd1-41a3-b2dd-68b5d0b338a9
b9ececef-5a0c-4294-be16-334e8017f60f
8e635367-35ff-42f9-bb21-88ae9db8be8d
64636605-10da-4576-bf21-a265cc1f7d9c


{'b02': 48d30acd-a378-4f2a-89b3-d73430c0f29e,
 'b03': 200ed06c-fcd1-41a3-b2dd-68b5d0b338a9,
 'b04': b9ececef-5a0c-4294-be16-334e8017f60f,
 'b08': 8e635367-35ff-42f9-bb21-88ae9db8be8d,
 'scl': 64636605-10da-4576-bf21-a265cc1f7d9c}